### introduction to Bioinformatics
### homework 2
### Name : Taraneh Khosrojerdi
### Student Id : 400104929

In [3]:
import numpy as np
import subprocess
import random

In [4]:
# FTP URL of the FASTQ file you want to download
ftp_url = 'ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR390/SRR390728/SRR390728_1.fastq.gz'

# Destination path where the downloaded file will be saved
destination_path = 'SRR390728_1.fastq.gz'

# Download data using wget
try:
    subprocess.run(['wget', ftp_url, '-O', destination_path], check=True)
    print(f"Downloaded data for SRR390728_1.fastq.gz")
except subprocess.CalledProcessError as e:
    print(f"Error: {e}")

Downloaded data for SRR390728_1.fastq.gz


In [5]:
#Now You have to read from SRR390728_1.fastq.gz file and save all of this file sequence in concatenated_sequence
#hint : use gzip library and read about fastq files and how they save genomes sequence
import gzip

concatenated_sequences = ""
with gzip.open("SRR390728_1.fastq.gz", 'rt') as f:
  reads = f.readlines()
  for i in range(1, len(reads), 4):
    r = reads[i].strip()
    if 'N' not in r:
      concatenated_sequences += r


In [6]:
#because genome is too long we just get first 35000 of genomes sequence and work with this genome after it
#so only run this cell
refrence_genome = concatenated_sequences[0:35000]

### just run the cell to create short reads from genome

In [7]:

# Minimum length of each short read
min_read_length = 90

# Function to generate random short reads
def generate_random_short_reads(sequence, min_length):
    short_reads = []
    remaining_sequence = sequence

    while len(remaining_sequence) >= min_length:
        read_length = random.randint(min(min_length, len(remaining_sequence)), min(len(remaining_sequence), 700))
        short_read = remaining_sequence[:read_length]
        short_reads.append(short_read)
        remaining_sequence = remaining_sequence[read_length:]

    return short_reads

def mutate(input):
    valid_inputs = ['A', 'C', 'T', 'G']
    num = random.randint(1, 1000)
    valid_inputs = [element for element in valid_inputs if element not in [input]]

    if num < 40 :
        shuffled_indices = list(range(len(valid_inputs)))
        random.shuffle(shuffled_indices)
        return valid_inputs[shuffled_indices[0]]
    return input


# Generate random short reads
copy_num = 4
short_reads = generate_random_short_reads(refrence_genome, min_read_length)
for i in range (copy_num):
    short_reads = short_reads + generate_random_short_reads(refrence_genome, min_read_length)


shuffled_indices = list(range(len(short_reads)))
random.shuffle(shuffled_indices)

# Create a new list to store shuffled short reads
short_reads = [short_reads[i] for i in shuffled_indices]

#mutations
for i in range (len(short_reads)):
    for j in range (len(short_reads[i])):
        mutate_char = mutate(short_reads[i][j])
        if mutate_char != short_reads[i][j]:
            if j != len(short_reads[i])-1:
                short_reads[i] = short_reads[i][:j]+mutate_char+short_reads[i][j+1:]
            else :
                short_reads[i] = short_reads[i][:j]+mutate_char




### just run the code below (it's just for calculating local alignment score with refrence genome)

In [8]:

def smith_waterman(sequence1, sequence2, match=2, mismatch=-1, gap_open=-2, gap_extend=-1):
    len_seq1, len_seq2 = len(sequence1), len(sequence2)
    score_matrix = np.zeros((len_seq1 + 1, len_seq2 + 1), dtype=int)

    for i in range(1, len_seq1 + 1):
        for j in range(1, len_seq2 + 1):
            match_mismatch_score = match if sequence1[i - 1] == sequence2[j - 1] else mismatch
            diagonal_score = score_matrix[i - 1][j - 1] + match_mismatch_score
            gap_up_score = score_matrix[i - 1][j] + gap_extend if score_matrix[i - 1][j] > 0 else gap_open
            gap_left_score = score_matrix[i][j - 1] + gap_extend if score_matrix[i][j - 1] > 0 else gap_open

            score_matrix[i][j] = max(0, diagonal_score, gap_up_score, gap_left_score)

    # Find the maximum score in the score matrix
    max_score = np.max(score_matrix)

    return max_score


In [9]:
# Print 10 samples of short reads with length
print(len(short_reads))
for i in range(10):
  index = random.randrange(0, len(short_reads) - 1)
  print(short_reads[i], len(short_reads[i]))

430
AGACCGGGGACAAAACAACATCATGGTCCAACTTCTGGAGAGAGCGCATCATTTTTGATCTCCTGGCCGGACTGTAGAGGGCGCATCCTTTTTGACCTCCTGCGCGGAGTGGTCAAAACAGCATCATGGTTCAACTTCTGGTAACAATGTAGAGAGCGGATCCTTTTTGAGCTCCTGGGCCGAACAACAACATGTTACAATTTATGGTAACATGTCAGCATCGTACAACTTCTGGTAACATGTCAGTGGGCGACAAGAATCTAGAGGGCGCATCCTTTTTGATCTCCTGGCATTTAAGAATGTAGAGAGCGCATCCTTTTTGATCTGATCAAAAAGGATGCGCTCTCTACATTCTTAAATGCGCATTTAAGAATGTAGAGAGCGCATCCTTTTTTATCAACTCCTGGTAACATGTCA 417
TGCCAGCGTCTTTACCTCCTTCCCACAACCAACAGTAAAAACACTACTCAAGATGGAAACTTCGTTACAGTTAACAGTGTTTGGCTTTTTTTCTTTTTGTGTTTTCAAGTTGCCATGGGGAAGAAGGAGGCTCTTTACTCGTTCAATCATTAGATTTTACGAACTTTTTAAAAGTTAAAATCGACTGTAACTAAGTTTCCATCTTCATTAGTGTTTCAACTGAAACTAAGGTTCTATCTT 240
CAGAAGTGCAAGGCTGTAGTGAGGCATGACACCCAGAAGTTCAAGGCTGTAGTGAGCCATGATCATAAGATCAGCATTGCAGTCTCATCTCTTTGGAGGGTTAGCCTGGGCAACGGAGTGAGTCCCTGTCTTAAAAACGGCGTGGCGCCCTCCAGCTGGGCGTGAACCAGGCACCTGGCCTGGCGTCCTCCAGCTGGGTGTGCACCAGGCACAAACAAACAAACAAACAAACAAAAACGTACTACACAAACAAACAAACAAACAAACAAAAACCTACTACACCTACTACAGGATTGATGCCAAACAAATGTTAAACAAGATTGATGTCA

In [ ]:
def find_max_overlap(strings):
    max_overlap = 0
    max_overlap_index = -1

    base_seq = strings[0]
    for i in range(1, len(strings)):
            overlap = calculate_overlap(base_seq, strings[i])

            if overlap > max_overlap:
                max_overlap = overlap
                max_overlap_index = i

    return max_overlap_index, max_overlap

def calculate_overlap(string1, string2):
    max_overlap = min(len(string1), len(string2))

    for i in range(max_overlap, 0, -1):
        if string1.endswith(string2[:i]):
            return i

    return 0


In [ ]:
def merge(T, j, l):
  st2 = T[j]
  T[0] += T[j][l:]
  T.pop(j)

In [ ]:

# Function to perform greedy overlap-based assembly
def heuristic_assemble(short_reads):
    #TODO
    T = short_reads
    while len(T) > 1 and len(T[0]) < 1500:
      i, l = find_max_overlap(T)
      merge(T, i, l)

    return T[0]

# Perform greedy overlap-based assembly
short_reads_copy = short_reads
assembled_sequence = heuristic_assemble(short_reads_copy)
print(len(assembled_sequence))


1538


### Your alignment score should be at least 1250

In [ ]:
# Calculate the local alignment score using Smith-Waterman algorithm
alignment_score = smith_waterman(refrence_genome, assembled_sequence)

# Print the local alignment score
print("Local Alignment Score:", alignment_score)

Local Alignment Score: 1736


In [12]:
import copy

def generate_k_mers(reads, k):
    k_mers = []
    for read in reads:
        for i in range(len(read) - k + 1):
            k_mers.append(read[i:i+k])
    return k_mers

def construct_graph(strings):
    g = {}
    for s in strings:
        prefix = s[:-1]
        suffix = s[1:]
        if prefix not in g:
          g[prefix] = []
        g[prefix].append(suffix)

    return g

def find_assembly(k_mers, g, seq):
  for item in g.items():
    assembly = item[0]
    curr_node = item[0]
    g_copy = copy.copy(g)
    while True:
      if curr_node not in g_copy or len(g_copy[curr_node]) == 0:
        break
      tmp_node = g_copy[curr_node][0]
      assembly += tmp_node[-1]
      g_copy[curr_node].remove(tmp_node)
      curr_node = tmp_node

    if len(assembly) > 2500:
      seq = assembly
      break
  return seq

In [17]:
def perform_de_novo_assembly(short_reads, k=25):
  k_mers = generate_k_mers(short_reads, k)
  g = construct_graph(k_mers)
  seq = ''

  return find_assembly(k_mers, g, seq)

de_brujin_assembled_sequence = perform_de_novo_assembly(short_reads)
print(len(de_brujin_assembled_sequence))

2608


### Your alignment score should be at least 3800

In [19]:
alignment_score = smith_waterman(refrence_genome, de_brujin_assembled_sequence)
# Print the local alignment score
print("Local Alignment Score:", alignment_score)


Local Alignment Score: 4710
